In [1]:
from selenium import webdriver
import time
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 

In [2]:
url = "https://www.magicbricks.com/property-for-rent/residential-real-estate?bedroom=1,2,3,4&proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Service-Apartment,Residential-House,Villa&cityName=Hyderabad"

In [19]:
# Initialize Selenium driver
driver = webdriver.Chrome()
driver.get(url)


In [20]:
import time
old_height = driver.execute_script('return document.body.scrollHeight') # 1000


while True:

    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)') # 40
    time.sleep(4)
    new_height = driver.execute_script('return document.body.scrollHeight') # 40 

    if new_height == old_height: 
        break
    old_height = new_height 


html = driver.page_source

In [21]:
soup = BeautifulSoup(html)

In [29]:
for i in soup.find_all("div",class_="mb-srp__list"):
    print(i.text)

12+ PhotosPosted:   YesterdayAgent: UpbricksOperating Since: 20004 BHK Flat  for Rent in Kokapet, Outer Ring Road, HyderabadSonthalia Fortune Sky VillasFurnishingFurnishedBathroom5Tenant PreferredFamilyAvailabilityImmediatelyCarpet Area7333 sqftFloor8 out of 28facingEastoverlookingMain RoadBathroom5Balcony1East Facing Property7333 sft 4bhk flat for sale in DSR Sonthalia The monthly maintenance is Rs 29000/-.₹3.5 Lac Security DepositContact Agent Get Phone No. 
7+ PhotosPosted:   YesterdayAProperty StudioCRISIL Rating: 3 Know More 1 BHK Flat  for Rent in Kondapur, HyderabadKondapurFurnishingFurnishedBathroom1Tenant PreferredBachelors/FamilyAvailabilityImmediatelyCarpet Area450 sqftFloor4 out of 5facingNorthoverlookingMain RoadBathroom1Balcony1Kondapur in Hyderabad is a key IT hub and residential area, known for its proximity to HITECH City and the presence of major tech firms, offering ready apartments and convenient access to amenities like malls, hospitals, and school The monthly main

In [61]:
BHK = []
Type = []
Location = []
Name = []
Furnishing = []
Bathroom = []
Tenant = []
Availability = []
Area = []
Floor = []
Maintenance = []
Rent = []


for i in soup.find_all("div",class_="mb-srp__list"):

    text = i.text

    # finding the number of bedrooms
    bhk = re.findall(r"(\d)\s?BHK",text)
    if bhk:
        BHK.append(bhk[0])
    else:
        BHK.append(np.nan)

    # type of the property
    type = re.findall(r"(House|Flat|Villa|Apartment)",text)
    if type:
        Type.append(type[0])
    else:
        Type.append(np.nan)

    # finding the location of the property
    location_regex = r"(?i)(?:for rent|rent)\s+in\s+(.+?)\s*(?:,?\s*Hyderabad|$)"
    l = re.findall(location_regex, text, flags=re.IGNORECASE)

    if l:
        # Clean up extracted location
        location = l[0].strip(',. ').replace('  ', ' ')
        Location.append(location)
    else:
        Location.append(np.nan)

    # Name of the society/property
    name = re.findall(r'Hyderabad\s*(.*?)(?=\s*(?:Furnishing|Super|Bathroom|Tenant|Availability))', text)
    if name:
        Name.append(name[0])
    else:
        Name.append(np.nan)

    # Furnishing status
    f = re.findall(r"(Semi-Furnished|Furnished|Unfurnished)",text)
    if f:
        Furnishing.append(f[0])
    else:
        Furnishing.append(np.nan)

    # number of bathrooms
    b = re.findall(r"Bathroom\s?(\d)",text)
    if b:
        Bathroom.append(b[0])
    else:
        Bathroom.append(np.nan)

    # Tenant prefered
    tenant = re.findall(r"(Bachelors/Family|Family|Bachelors)",text)
    if tenant:
        Tenant.append(tenant[0])
    else:
        Tenant.append(np.nan)

    # Availability of the property
    a = re.findall(r"Availability\s?(.*?)(?=Super|Carpet)",text)
    if a:
        Availability.append(a[0])
    else:
        Availability.append(np.nan)

    # Area of the property
    area = re.findall(r"Area(\d+)\s?sqft",text)
    if area:
        Area.append(area[0])
    else:
        Area.append(np.nan)

    # floor number
    floor = re.findall(r"Floor(\d)",text)
    if floor:
        Floor.append(floor[0])
    else:
        Floor.append(np.nan)

    # Maintenance charges
    m = re.findall(r"maintenance is Rs (\d+)",text)
    if m:
        Maintenance.append(m[0])
    else:
        Maintenance.append(np.nan)

    rent = i.find("div",class_="mb-srp__card__price--amount")
    if rent:
        Rent.append(rent.text)
    else:
        Rent.append(np.nan)

In [57]:
dict_1 = {"Type":Type,"Bedrooms":BHK,"Location":Location,"Name_of_the_Society":Name,"Furnishing_Status":Furnishing,
          "Bathrooms":Bathroom,"Tenant_Prefered":Tenant,"Availability":Availability,"Area_in_sqft":Area,"Floor":Floor,
          "Maintenance_charges":Maintenance,"Rent":Rent}

In [58]:
# Creating a DataFrame out of the scraped data
df = pd.DataFrame(dict_1)

In [59]:
df

,Type,Bedrooms,Location,Name_of_the_Society,Furnishing_Status,Bathrooms,Tenant_Prefered,Availability,Area_in_sqft,Floor,Maintenance_charges,Rent
0,Flat,4,"Kokapet, Outer Ring Road",Sonthalia Fortune Sky Villas,Furnished,5,Family,Immediately,7333,8,29000,₹3.5 Lac
1,Flat,1,Kondapur,Kondapur,Furnished,1,Bachelors/Family,Immediately,450,4,1000,"₹25,000"
2,Flat,1,Kondapur,Kondapur,Furnished,1,Bachelors/Family,From Jun '25,700,5,1500,"₹26,500"
3,Flat,3,Kondapur,GHR Titania,Semi-Furnished,3,Bachelors,Immediately,1788,5,NaN,"₹55,000"
4,Flat,3,"Kokapet, Outer Ring Road",Prestige Tranquil,Semi-Furnished,3,Family,Immediately,1166,3,4,"₹65,000"
...,...,...,...,...,...,...,...,...,...,...,...,...
685,NaN,1,My home jewel madinaguda,Hyderabad,Furnished,1,Bachelors/Family,Immediately,690,4,NaN,"₹12,000"
686,Apartment,2,Prakasham Panthulu Nagar,,Furnished,2,Bachelors,Immediately,780,1,NaN,"₹14,000"
687,House,2,Yellampet,,Unfurnished,1,Bachelors,Immediately,1300,NaN,500,"₹12,000"
688,Villa,3,Thirusankalp villas nizampet,,Semi-Furnished,3,Bachelors,Immediately,1800,NaN,NaN,"₹40,000"


In [60]:
# Saving the DataFrame as a csv file
rental_df = df.to_csv("Rental_data.csv")

In [18]:
driver.quit()